In [72]:
import pandas as pd
import pdfplumber
import os
import numpy as np
import re
from tqdm.notebook import tqdm
from lxml import etree

In [4]:
ocrdata = pd.read_csv('../data/covid19wob_files_df.tar.xz')

In [92]:
def clean_split(text):
    splitted = re.split('\W+', text)
    cleaned = [word.lower() for word in splitted if word]
     
    return cleaned

def count_words(text):
    
    if not isinstance(text, str):
        return 0

    corpus = clean_split(text)
    
    return len(corpus)

def count_characters(text):
    
    if not isinstance(text, str):
        return 0
    
    corpus = clean_split(text)
    
    return sum([len(word) for word in corpus])

In [70]:
ocrdata['nr_words'] = ocrdata['text'].map(lambda x: count_words(x))
ocrdata['nr_chars'] = ocrdata['text'].map(lambda x: count_characters(x))

In [87]:
first = set(clean_split(ocrdata['text'][0]))

In [102]:
import csv

dirpath = '../data/XMLs//'

headers = ['name', 'page', 'text']

with open('output.csv', "w", encoding="utf-8") as writefile:
    
    writer = csv.writer(writefile)
    writer.writerow(headers)
    
    for xmlfile in os.listdir(dirpath):

        root = etree.parse(dirpath+xmlfile)
        pages = root.xpath('//page')

        for pagenumber, page in enumerate(pages):
            
            writer.writerow([xmlfile.strip('.xml'), pagenumber+1, ' '.join(page.xpath('text/text()'))])


In [108]:
import gzip

with gzip.open('wobcovid_xml.csv.gz', 'wb') as f_out:
    f_in = open('output.csv', 'rb')
    f_out.writelines(f_in)
    f_in.close()

In [114]:
df = pd.read_csv('wobcovid_xml.csv.gz', error_bad_lines=False, chunksize=1000)

ParserError: Error tokenizing data. C error: out of memory